In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

In [3]:
app = dash.Dash()

In [4]:
if __name__ == '__main__':
    app.run_server(port = 4050)

[2024-05-11 17:23:59,453] ERROR in app: Exception on /_alive_b102d508-128f-4f5a-839f-fa3242eb1c44 [GET]
Traceback (most recent call last):
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 878, in full_dispatch_request
    rv = self.preprocess_request()
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 1253, in preprocess_request
    rv = self.ensure_sync(before_func)()
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/dash/dash.py", line 1391, in _setup_server
    _validate.validate_layout(self.layout, self._layout_value())
  File "/Users/pps

In [5]:
from chessdotcom import get_player_game_archives, Client

# Set custom User-Agent header for the client
Client.request_config["headers"]["User-Agent"] = (
    "My Python Application. "
    "Contact me at email@example.com"
)

# Retrieve game archives for the player "ppsahusg"
gamesUrl = get_player_game_archives("hikaru")

import pandas as pd
import numpy as np
import json
import pprint
from pprint import pprint
import _collections_abc
from collections.abc import MutableMapping
import sys

if sys.version_info.major == 3 and sys.version_info.minor >= 10:
    import collections
    setattr(collections, "MutableMapping", collections.abc.MutableMapping)

import chess.pgn

import requests
import json
import re

def create_dataset(api_url):
    api_url = api_url

    headers = {
        "User-Agent": "My Chess Analysis Tool Application. Contact me at pratyush.sahu@csu.fullerton.edu"
    }

    response = requests.get(api_url, headers = headers)

    if response.status_code == 200:
        try:
            data = response.json()
            games = []

            for game in data['games']:
                termination = None
                eco_url = None
                utc_date = None

                if 'pgn' in game:
                    termination_match = re.search(r'\[Termination "(.*?)"\]', game['pgn'])
                    eco_match = re.search(r'\[ECOUrl "(.*?)"\]', game['pgn'])
                    utc_date_match = re.search(r'\[UTCDate "(.*?)"\]', game['pgn'])

                    if termination_match:
                        termination = termination_match.group(1)
                    if eco_match:
                        eco_url = eco_match.group(1)
                    if utc_date_match:
                        utc_date = utc_date_match.group(1)

                games.append({
                    "White": game["white"]["username"],
                    "Black": game["black"]["username"],
                    "Date": game["end_time"],
                    "URL": game["url"],
                    "Link": eco_url,
                    "Termination": termination,
                    "TimeControl": game["time_control"],
                    "UTCDate": utc_date
                })
            return games

        except json.JSONDecodeError:
            print("Error decoding JSON: The response is not valid JSON.")
            return
    else:
        print(f"Request error. Status code: {response.status_code}")
        return

games = create_dataset("https://api.chess.com/pub/player/Hikaru/games/2024/01")
df = pd.DataFrame(games)

In [6]:
def categorize_result(value):
    if 'Hikaru' in value:
        return 'Win'
    elif 'draw' in value:
        return 'Draw'
    else:
        return 'Loss'

df['Termination'] = df['Termination'].apply(categorize_result)
df['Piece_Color'] = df.apply(lambda row: 'White' if 'Hikaru' in row['White'] else 'Black' if 'Hikaru' in row['Black'] else None, axis=1)
print(df)

df['Modality'] = df.apply(lambda row: 'bullet' if '60' in row['TimeControl'] else
                                    ('blitz' if '180' in row['TimeControl'] else
                                     ('blitz' if '300' in row['TimeControl'] else
                                      ('rapid' if '600' in row['TimeControl'] else None))), axis=1)

df['Link'].unique()

# Extract opening name from the URL link and remove trailing digits and hyphens
df['Opening'] = df['Link'].apply(lambda x: re.search(r'/openings/([^/]+)', x).group(1))
df['Opening'] = df['Opening'].str.replace(r'-\d.*$', '', regex=True)

df['Opening'].nunique()

new_df = df.drop(columns=['White', 'Black', 'Date', 'TimeControl', 'Link'])

new_df = df
# print(new_df)

new_df = new_df[new_df['Modality'] == 'blitz']

def group_openings(opening):
    if 'Modern' in opening:
        return 'Modern Defense'
    elif 'Vienna' in opening:
        return 'Vienna Game'
    elif 'English' in opening:
        return 'English Opening'
    elif 'Giuoco-Piano' in opening:
        return 'Giuoco Piano'
    elif 'Caro-Kann' in opening:
        return 'Caro-Kann Defense'
    elif 'Queens-Pawn' in opening:
        return 'Queens Pawn Opening'
    elif 'Sicilian' in opening:
        return 'Sicilian Defense'
    elif 'Nimzowitsch' in opening:
        return 'Nimzowitsch Defense'
    elif 'Kings' in opening:
        return 'Kings Pawn Opening'
    elif 'Indian' in opening:
        return 'Indian Game'
    elif 'Closed-Sicilian' in opening:
        return 'Closed Sicilian Defense'
    elif 'Italian-Game' in opening:
        return 'Italian_Game'
    else:
        return opening

new_df['Grouped_Opening'] = new_df['Opening'].apply(group_openings)

                White          Black        Date  \
0              Hikaru  Gravity_Chess  1704133706   
1       Gravity_Chess         Hikaru  1704133905   
2     GiorgianBarbosa         Hikaru  1704211627   
3              Hikaru          Witik  1704212359   
4              dretch         Hikaru  1704212913   
...               ...            ...         ...   
1040        Snowflake         Hikaru  1706735112   
1041           Hikaru      Snowflake  1706735343   
1042        Snowflake         Hikaru  1706735699   
1043           Hikaru      Snowflake  1706735830   
1044        Snowflake         Hikaru  1706736147   

                                               URL  \
0      https://www.chess.com/game/live/97872578329   
1      https://www.chess.com/game/live/97872681559   
2      https://www.chess.com/game/live/97950132781   
3      https://www.chess.com/game/live/97950760747   
4      https://www.chess.com/game/live/97951339723   
...                                            ... 

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1636908453.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
new_df

,White,Black,Date,URL,Link,Termination,TimeControl,UTCDate,Piece_Color,Modality,Opening,Grouped_Opening
0,Hikaru,Gravity_Chess,1704133706,https://www.chess.com/game/live/97872578329,https://www.chess.com/openings/Closed-Sicilian...,Win,180,2024.01.01,White,blitz,Closed-Sicilian-Defense,Sicilian Defense
1,Gravity_Chess,Hikaru,1704133905,https://www.chess.com/game/live/97872681559,https://www.chess.com/openings/English-Opening...,Win,180,2024.01.01,Black,blitz,English-Opening-Great-Snake-Variation,English Opening
2,GiorgianBarbosa,Hikaru,1704211627,https://www.chess.com/game/live/97950132781,https://www.chess.com/openings/English-Opening...,Win,180+1,2024.01.02,Black,blitz,English-Opening-Great-Snake-Variation...3.Bg2-d6,English Opening
3,Hikaru,Witik,1704212359,https://www.chess.com/game/live/97950760747,https://www.chess.com/openings/English-Opening...,Win,180+1,2024.01.02,White,blitz,English-Opening-Kings-English-Variation,English Opening
4,dretch,Hikaru,1704212913,https://www.chess.com/game/live/97951339723,https://www.chess.com/openings/Owens-Defense-2.d4,Win,180+1,2024.01.02,Black,blitz,Owens-Defense,Owens-Defense
...,...,...,...,...,...,...,...,...,...,...,...,...
1040,Snowflake,Hikaru,1706735112,https://www.chess.com/game/live/100473784011,https://www.chess.com/openings/Sicilian-Defens...,Win,180,2024.01.31,Black,blitz,Sicilian-Defense-Kan-Maroczy-Bind-Formation,Sicilian Defense
1041,Hikaru,Snowflake,1706735343,https://www.chess.com/game/live/100474293187,https://www.chess.com/openings/Closed-Sicilian...,Win,180,2024.01.31,White,blitz,Closed-Sicilian-Defense-Grand-Prix-Schofman-Va...,Sicilian Defense
1042,Snowflake,Hikaru,1706735699,https://www.chess.com/game/live/100474374241,https://www.chess.com/openings/Modern-Defense-...,Win,180,2024.01.31,Black,blitz,Modern-Defense-with,Modern Defense
1043,Hikaru,Snowflake,1706735830,https://www.chess.com/game/live/100474888935,https://www.chess.com/openings/Reti-Opening-Ni...,Win,180,2024.01.31,White,blitz,Reti-Opening-Nimzo-Larsen-Variation,Reti-Opening-Nimzo-Larsen-Variation


In [8]:
termination_count = new_df['Termination'].value_counts()
termination_count

new_tc = pd.DataFrame(termination_count)
# new_tc['Results'] = termination_count.index
# new_tc['Counts'] = termination_count.values
new_tc

,count
Termination,
Win,624
Draw,74
Loss,59


In [9]:
# app.layout = html.Div([
#     html.H1(children = "Chess Analysis Engine Tool",
#             style = {
#                 'textAlign' : 'center',
#                 'color' : '#ff0000'
#             }
           
#            ),
#     html.Div(children = 'A perfect tool for you to analyse your chess games and become better!',
#             style = {
#                 'textAlign' : 'center',
#                 'color' : '#ff0000'
#             }
#             ),
#     dcc.Graph(
#         id = 'samplechart',
#         figure = {
#             'data' : [
#                 {'x': termination_count.index, 'y':termination_count.values, 'type':'bar', 'name':'first chart'}
#             ],
#             'layout' : {
#                 'title' : 'Win / Loss / Draw -  Bar Chart'
#             }
#         }
#     ),
#     dcc.Graph(
#         id = 'samplechart',
#         figure = {
#             'data' : [
#                 {'x': termination_count.index, 'y':termination_count.values, 'type':'bar', 'name':'first chart'}
#             ],
#             'layout' : {
#                 'title' : 'Win / Loss / Draw - Bar Chart'
#             }
#         }
#     )
# ])

In [10]:
# Create a column indicating whether each game resulted in a win (1 for victory, 0 otherwise)
new_df['Victory'] = new_df['Termination'].apply(lambda x: 1 if x == 'Win' else 0)

# Group the DataFrame by the grouped openings and calculate statistics
opening_statistics = new_df.groupby('Grouped_Opening').agg({'Victory': ['count', 'sum']})

print(opening_statistics)

# Rename the columns for clarity
opening_statistics.columns = ['Total_Games', 'Victories']

# Calculate the victory rate for each opening category
opening_statistics['Victory_Rate'] = opening_statistics['Victories'] / opening_statistics['Total_Games'] * 100

# Sort the statistics by the total number of games in descending order
opening_statistics = opening_statistics.sort_values(by='Total_Games', ascending=False)

# Print the statistics
print(opening_statistics)

                                                 Victory    
                                                   count sum
Grouped_Opening                                             
Alekhines-Defense-Exchange-Variation...6.Nc3-Be7       1   1
Alekhines-Defense-Normal-Variation                     5   5
Alekhines-Defense-Samisch-Attack                       2   2
Alekhines-Defense-Scandinavian-Variation               3   2
Alekhines-Defense-Two-Pawns-Lasker-Variation           1   0
...                                                  ...  ..
Trompowsky-Attack...3.Bf4-d5                           1   1
Trompowsky-Attack...3.Bxf6-exf6                        1   1
Trompowsky-Attack...3.e3-c5                            1   1
Vant-Kruijs-Opening                                    1   1
Vienna Game                                           32  26

[87 rows x 2 columns]
                                                 Total_Games  Victories  \
Grouped_Opening                                 

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1074106147.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
# Create a column indicating whether each game resulted in a win (1 for victory, 0 otherwise)
new_df['Victory'] = new_df['Termination'].apply(lambda x: 1 if x == 'Win' else 0)

# Group the DataFrame by the grouped colors and calculate statistics
color_opening_statistics = new_df.groupby('Piece_Color').agg({'Victory': ['count', 'sum']})

print(color_opening_statistics)

# Rename the columns for clarity
color_opening_statistics.columns = ['Total_Games', 'Victories']

# Calculate the victory rate for each color category
color_opening_statistics['Victory_Rate'] = color_opening_statistics['Victories'] / color_opening_statistics['Total_Games'] * 100

# Sort the statistics by the total number of games in descending order
color_opening_statistics = color_opening_statistics.sort_values(by='Total_Games', ascending=False)

# Print the statistics
print(color_opening_statistics)

            Victory     
              count  sum
Piece_Color             
Black           373  306
White           384  318
             Total_Games  Victories  Victory_Rate
Piece_Color                                      
White                384        318     82.812500
Black                373        306     82.037534


/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/322570639.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
top_10_openings = opening_statistics.head(10)

In [13]:
top_10_openings = opening_statistics.head(10)
top_10_openings

,Total_Games,Victories,Victory_Rate
Grouped_Opening,,,
Modern Defense,152,128,84.210526
Sicilian Defense,112,93,83.035714
Caro-Kann Defense,52,41,78.846154
English Opening,48,46,95.833333
Kings Pawn Opening,46,41,89.130435
Indian Game,35,24,68.571429
Vienna Game,32,26,81.250000
Nimzowitsch Defense,31,30,96.774194
Reti-Opening-Nimzo-Larsen-Variation,30,25,83.333333


In [14]:
# Group games by UTCDate and aggregate the number of victories for each date
statistics_data1 = new_df.groupby('UTCDate').agg({'Victory': 'sum'})

# Sort the statistics by the number of victories in descending order
statistics_data2 = statistics_data1.sort_values(by='Victory', ascending=False)
statistics_data1

,Victory
UTCDate,
2024.01.01,2
2024.01.02,54
2024.01.03,86
2024.01.05,57
2024.01.06,26
2024.01.09,34
2024.01.10,63
2024.01.13,2
2024.01.15,23


In [15]:
# Filter games where the termination is a draw, group by grouped openings, and count the occurrences
draws_by_opening = new_df[new_df['Termination'] == 'Draw'].groupby('Grouped_Opening').size()

# Sort the number of draws by opening in descending order
# draws_by_opening = draws_by_opening.sort_values(ascending=False)
draws_by_opening

Grouped_Opening
Alekhines-Defense-Scandinavian-Variation                                1
Alekhines-Defense-Two-Pawns-Lasker-Variation                            1
Bishops-Opening-Berlin-Defense                                          1
Bishops-Opening-Berlin-Defense...4.Nf3-d5                               1
Bishops-Opening-Berlin-Defense...5.Bb3-Bb4                              1
Bishops-Opening-Berlin-Spielmann-Attack                                 1
Caro-Kann Defense                                                       5
English Opening                                                         1
French-Defense-Classical-Steinitz-Boleslavsky-Variation...8.Qd2-O-O     1
French-Defense-Normal-Variation                                         1
Indian Game                                                             7
Kings Pawn Opening                                                      3
Modern Defense                                                         13
Nimzowitsch Defense   

In [16]:
win_by_piece = new_df.groupby('Piece_Color')['Victory'].sum()
black_count = win_by_piece[0]
white_count = win_by_piece[1]
win_by_piece

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1683409538.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1683409538.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Piece_Color
Black    306
White    318
Name: Victory, dtype: int64

In [17]:
app.layout = html.Div([
    html.H1(children = "Chess Analysis Engine Tool",
            style = {
                'textAlign' : 'center',
                'color' : '#ff0000'
            }
           
           ),
    html.Div(children = 'A perfect tool for you to analyse your chess games and become better!',
            style = {
                'textAlign' : 'center',
                'color' : '#ff0000'
            }
            ),
    dcc.Input(id = "input1", type="text", placeholder="", style={'center':'10px'}),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': termination_count.index, 'y':termination_count.values, 'type':'bar', 'name':'first chart'}
            ],
            'layout' : {
                'title' : 'Win / Loss / Draw -  Bar Chart'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': top_10_openings.index, 'y':top_10_openings['Victory_Rate'], 'type':'bar', 'name':'first chart'}
            ],
            'layout' : {
                'title' : 'Top 10 Most Openings Played'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': top_10_openings.index, 'y':top_10_openings['Total_Games'], 'type':'bar', 'name':'Total Games'},
                {'x': top_10_openings.index, 'y':top_10_openings['Victories'], 'type':'bar', 'name':'Victories'},
                {'x': top_10_openings.index, 'y':top_10_openings['Victory_Rate'], 'type':'bar', 'name':'Victory Rate'}
            ],
            'layout' : {
                'title' : 'Victory Rate Against Top 10 Openings'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': statistics_data1.index, 'y':statistics_data1['Victory'], 'type':'bar', 'name':'Total Games'}
            ],
            'layout' : {
                'title' : 'Wins on all dates of the month'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': draws_by_opening.index, 'y':draws_by_opening, 'type':'bar', 'name':'Total Games'}
            ],
            'layout' : {
                'title' : 'Draws Against All Openings'
            }
        }
    ),
    dcc.Graph(figure=go.Figure(data=[go.Pie(labels=['Black','White'], values=[black_count, white_count])])),
])

@app.callback(
    Output("pie-chart", "figure"))
def generate_chart():
    black_count = win_by_piece[0]
    white_count = win_by_piece[1]
    fig = go.Figure(data=[go.Pie(labels=['Win by Black','Win by White'], values=[black_count, white_count])])
    fig.update_layout(title="Wins by color")
    return fig

NameError: name 'Output' is not defined

In [18]:
if __name__ == '__main__':
    app.run_server(port = 4050)

In [19]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app.layout = html.Div([
    html.H1(children = "Chess Analysis Engine Tool",
            style = {
                'textAlign' : 'center',
                'color' : '#ff0000'
            }
           
           ),
    html.Div(children = 'A perfect tool for you to analyse your chess games and become better!',
            style = {
                'textAlign' : 'center',
                'color' : '#ff0000'
            }
            ),
    dcc.Input(id = "input1", type="text", placeholder="", style={'center':'10px'}),
    dcc.Graph(id = 'samplechart1', figure={}),
    dcc.Graph(id = 'samplechart2', figure={})
    # dcc.Graph(
    #     id = 'samplechart1',
    #     figure = {
    #         'data' : [
    #             {'x': termination_count.index, 'y':termination_count.values, 'type':'bar', 'name':'first chart'}
    #         ],
    #         'layout' : {
    #             'title' : 'Win / Loss / Draw -  Bar Chart'
    #         }
    #     }
    # ),
    # dcc.Graph(
    #     id = 'samplechart2',
    #     figure = {
    #         'data' : [
    #             {'x': top_10_openings.index, 'y':top_10_openings['Victory_Rate'], 'type':'bar', 'name':'first chart'}
    #         ],
    #         'layout' : {
    #             'title' : 'Top 10 Most Openings Played'
    #         }
    #     }
    # )
])
@app.callback(
    [Output(component_id='samplechart1', component_property='figure1'),
     Output(component_id='samplechart2', component_property='figure2')],
    [Input(component_id='input1', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    dff = new_tc
    figure1 = px.bar(dff,
                 color="smoker", barmode="group")
    dff2 = top_10_openings
    figure2 = px.bar(dff2, x=top_10_openings.index, y=top_10_openings['Victory_Rate'], 
                 color="smoker", barmode="group")
    return fig1, fig2
   


In [20]:
if __name__ == '__main__':
    app.run_server(port = 5050)

[2024-05-11 17:24:11,012] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/Users/ppsahu/Downloads/python-dashboard/lib/python3.10/site-packages/dash/dash.py", line 1352, in dispatch
    ctx.run(
  File "/Users/ppsahu/Downloads/p

None


In [21]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

from chessdotcom import get_player_game_archives, Client

# Set custom User-Agent header for the client
Client.request_config["headers"]["User-Agent"] = (
    "My Python Application. "
    "Contact me at email@example.com"
)

# Retrieve game archives for the player "ppsahusg"
gamesUrl = get_player_game_archives("hikaru")

import pandas as pd
import numpy as np
import json
import pprint
from pprint import pprint
import _collections_abc
from collections.abc import MutableMapping
import sys

if sys.version_info.major == 3 and sys.version_info.minor >= 10:
    import collections
    setattr(collections, "MutableMapping", collections.abc.MutableMapping)

import chess.pgn

import requests
import json
import re

def create_dataset(api_url):
    api_url = api_url

    headers = {
        "User-Agent": "My Chess Analysis Tool Application. Contact me at pratyush.sahu@csu.fullerton.edu"
    }

    response = requests.get(api_url, headers = headers)

    if response.status_code == 200:
        try:
            data = response.json()
            games = []

            for game in data['games']:
                termination = None
                eco_url = None
                utc_date = None

                if 'pgn' in game:
                    termination_match = re.search(r'\[Termination "(.*?)"\]', game['pgn'])
                    eco_match = re.search(r'\[ECOUrl "(.*?)"\]', game['pgn'])
                    utc_date_match = re.search(r'\[UTCDate "(.*?)"\]', game['pgn'])

                    if termination_match:
                        termination = termination_match.group(1)
                    if eco_match:
                        eco_url = eco_match.group(1)
                    if utc_date_match:
                        utc_date = utc_date_match.group(1)

                games.append({
                    "White": game["white"]["username"],
                    "Black": game["black"]["username"],
                    "Date": game["end_time"],
                    "URL": game["url"],
                    "Link": eco_url,
                    "Termination": termination,
                    "TimeControl": game["time_control"],
                    "UTCDate": utc_date
                })
            return games

        except json.JSONDecodeError:
            print("Error decoding JSON: The response is not valid JSON.")
            return
    else:
        print(f"Request error. Status code: {response.status_code}")
        return

games = create_dataset("https://api.chess.com/pub/player/Hikaru/games/2024/01")
df = pd.DataFrame(games)

def categorize_result(value):
    if 'Hikaru' in value:
        return 'Win'
    elif 'draw' in value:
        return 'Draw'
    else:
        return 'Loss'

df['Termination'] = df['Termination'].apply(categorize_result)
df['Piece_Color'] = df.apply(lambda row: 'White' if 'Hikaru' in row['White'] else 'Black' if 'Hikaru' in row['Black'] else None, axis=1)
print(df)

df['Modality'] = df.apply(lambda row: 'bullet' if '60' in row['TimeControl'] else
                                    ('blitz' if '180' in row['TimeControl'] else
                                     ('blitz' if '300' in row['TimeControl'] else
                                      ('rapid' if '600' in row['TimeControl'] else None))), axis=1)

df['Link'].unique()

# Extract opening name from the URL link and remove trailing digits and hyphens
df['Opening'] = df['Link'].apply(lambda x: re.search(r'/openings/([^/]+)', x).group(1))
df['Opening'] = df['Opening'].str.replace(r'-\d.*$', '', regex=True)

df['Opening'].nunique()

new_df = df.drop(columns=['White', 'Black', 'Date', 'TimeControl', 'Link'])

new_df = df
# print(new_df)

new_df = new_df[new_df['Modality'] == 'blitz']

def group_openings(opening):
    if 'Modern' in opening:
        return 'Modern Defense'
    elif 'Vienna' in opening:
        return 'Vienna Game'
    elif 'English' in opening:
        return 'English Opening'
    elif 'Giuoco-Piano' in opening:
        return 'Giuoco Piano'
    elif 'Caro-Kann' in opening:
        return 'Caro-Kann Defense'
    elif 'Queens-Pawn' in opening:
        return 'Queens Pawn Opening'
    elif 'Sicilian' in opening:
        return 'Sicilian Defense'
    elif 'Nimzowitsch' in opening:
        return 'Nimzowitsch Defense'
    elif 'Kings' in opening:
        return 'Kings Pawn Opening'
    elif 'Indian' in opening:
        return 'Indian Game'
    elif 'Closed-Sicilian' in opening:
        return 'Closed Sicilian Defense'
    elif 'Italian-Game' in opening:
        return 'Italian_Game'
    else:
        return opening

new_df['Grouped_Opening'] = new_df['Opening'].apply(group_openings)

termination_count = new_df['Termination'].value_counts()
termination_count

new_tc = pd.DataFrame(termination_count)
# new_tc['Results'] = termination_count.index
# new_tc['Counts'] = termination_count.values
new_tc

# Create a column indicating whether each game resulted in a win (1 for victory, 0 otherwise)
new_df['Victory'] = new_df['Termination'].apply(lambda x: 1 if x == 'Win' else 0)

# Group the DataFrame by the grouped openings and calculate statistics
opening_statistics = new_df.groupby('Grouped_Opening').agg({'Victory': ['count', 'sum']})

print(opening_statistics)

# Rename the columns for clarity
opening_statistics.columns = ['Total_Games', 'Victories']

# Calculate the victory rate for each opening category
opening_statistics['Victory_Rate'] = opening_statistics['Victories'] / opening_statistics['Total_Games'] * 100

# Sort the statistics by the total number of games in descending order
opening_statistics = opening_statistics.sort_values(by='Total_Games', ascending=False)

# Print the statistics
print(opening_statistics)

# Create a column indicating whether each game resulted in a win (1 for victory, 0 otherwise)
new_df['Victory'] = new_df['Termination'].apply(lambda x: 1 if x == 'Win' else 0)

# Group the DataFrame by the grouped colors and calculate statistics
color_opening_statistics = new_df.groupby('Piece_Color').agg({'Victory': ['count', 'sum']})

print(color_opening_statistics)

# Rename the columns for clarity
color_opening_statistics.columns = ['Total_Games', 'Victories']

# Calculate the victory rate for each color category
color_opening_statistics['Victory_Rate'] = color_opening_statistics['Victories'] / color_opening_statistics['Total_Games'] * 100

# Sort the statistics by the total number of games in descending order
color_opening_statistics = color_opening_statistics.sort_values(by='Total_Games', ascending=False)

# Print the statistics
print(color_opening_statistics)

top_10_openings = opening_statistics.head(10)

# Group games by UTCDate and aggregate the number of victories for each date
statistics_data1 = new_df.groupby('UTCDate').agg({'Victory': 'sum'})

# Sort the statistics by the number of victories in descending order
statistics_data2 = statistics_data1.sort_values(by='Victory', ascending=False)

# Filter games where the termination is a draw, group by grouped openings, and count the occurrences
draws_by_opening = new_df[new_df['Termination'] == 'Draw'].groupby('Grouped_Opening').size()

# Sort the number of draws by opening in descending order
# draws_by_opening = draws_by_opening.sort_values(ascending=False)

win_by_piece = new_df.groupby('Piece_Color')['Victory'].sum()
black_count = win_by_piece[0]
white_count = win_by_piece[1]

app.layout = html.Div([
    html.H1(children = "Chess Analysis Engine Tool",
            style = {
                'textAlign' : 'center',
                'color' : '#ff0000'
            }
           
           ),
    html.Div(children = 'A perfect tool for you to analyse your chess games and become better!',
            style = {
                'textAlign' : 'center',
                'color' : '#ff0000'
            }
            ),
    dcc.Input(id = "input1", type="text", placeholder="", style={'center':'10px'}),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': termination_count.index, 'y':termination_count.values, 'type':'bar', 'name':'first chart'}
            ],
            'layout' : {
                'title' : 'Win / Loss / Draw -  Bar Chart'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': top_10_openings.index, 'y':top_10_openings['Victory_Rate'], 'type':'bar', 'name':'first chart'}
            ],
            'layout' : {
                'title' : 'Top 10 Most Openings Played'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': top_10_openings.index, 'y':top_10_openings['Total_Games'], 'type':'bar', 'name':'Total Games'},
                {'x': top_10_openings.index, 'y':top_10_openings['Victories'], 'type':'bar', 'name':'Victories'},
                {'x': top_10_openings.index, 'y':top_10_openings['Victory_Rate'], 'type':'bar', 'name':'Victory Rate'}
            ],
            'layout' : {
                'title' : 'Victory Rate Against Top 10 Openings'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': statistics_data1.index, 'y':statistics_data1['Victory'], 'type':'bar', 'name':'Total Games'}
            ],
            'layout' : {
                'title' : 'Wins on all dates of the month'
            }
        }
    ),
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [
                {'x': draws_by_opening.index, 'y':draws_by_opening, 'type':'bar', 'name':'Total Games'}
            ],
            'layout' : {
                'title' : 'Draws Against All Openings'
            }
        }
    ),
    dcc.Graph(figure=go.Figure(data=[go.Pie(labels=['Black','White'], values=[black_count, white_count])])),
])

@app.callback(
    Output("pie-chart", "figure"))
def generate_chart():
    black_count = win_by_piece[0]
    white_count = win_by_piece[1]
    fig = go.Figure(data=[go.Pie(labels=['Win by Black','Win by White'], values=[black_count, white_count])])
    fig.update_layout(title="Wins by color")
    return fig

if __name__ == '__main__':
    app.run_server(port = 5050)

                White          Black        Date  \
0              Hikaru  Gravity_Chess  1704133706   
1       Gravity_Chess         Hikaru  1704133905   
2     GiorgianBarbosa         Hikaru  1704211627   
3              Hikaru          Witik  1704212359   
4              dretch         Hikaru  1704212913   
...               ...            ...         ...   
1040        Snowflake         Hikaru  1706735112   
1041           Hikaru      Snowflake  1706735343   
1042        Snowflake         Hikaru  1706735699   
1043           Hikaru      Snowflake  1706735830   
1044        Snowflake         Hikaru  1706736147   

                                               URL  \
0      https://www.chess.com/game/live/97872578329   
1      https://www.chess.com/game/live/97872681559   
2      https://www.chess.com/game/live/97950132781   
3      https://www.chess.com/game/live/97950760747   
4      https://www.chess.com/game/live/97951339723   
...                                            ... 

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1026112870.py:149: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1026112870.py:160: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/08/kc3tgyx54mxbp7rwxkkx67jh0000gn/T/ipykernel_46151/1026112870.py:180: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in